In [21]:
import wikipedia

In [22]:
movie = wikipedia.page('List_of_Telugu_films_of_2013')

In [38]:
print movie.section('January-june')

None


In [1]:
import requests

In [2]:
r = requests.get('http://www.omdbapi.com/?t=The+Avengers&plot=full&r=json')

In [3]:
r.json()

{u'Actors': u'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth',
 u'Awards': u'Nominated for 1 Oscar. Another 31 wins & 66 nominations.',
 u'Country': u'USA',
 u'Director': u'Joss Whedon',
 u'Genre': u'Action, Adventure, Sci-Fi',
 u'Language': u'English, Russian',
 u'Metascore': u'69',
 u'Plot': u"Nick Fury is director of S.H.I.E.L.D, an international peace keeping agency. The agency is a who's who of Marvel Super Heroes, with Iron Man, The Incredible Hulk, Thor, Captain America, Hawkeye and Black Widow. When global security is threatened by Loki and his cohorts, Nick Fury and his team will need all their powers to save the world from disaster.",
 u'Poster': u'http://ia.media-imdb.com/images/M/MV5BMTk2NTI1MTU4N15BMl5BanBnXkFtZTcwODg0OTY0Nw@@._V1_SX300.jpg',
 u'Rated': u'PG-13',
 u'Released': u'04 May 2012',
 u'Response': u'True',
 u'Runtime': u'143 min',
 u'Title': u'The Avengers',
 u'Type': u'movie',
 u'Writer': u'Joss Whedon (screenplay), Zak Penn (story), Joss Whedon (s

In [76]:
r1 = requests.get('http://www.omdbapi.com/?t=Black&plot=full&r=json')

In [77]:
movieHeaders = list(r.json().keys())
print movieHeaders

[u'Plot', u'Rated', u'tomatoImage', u'Title', u'DVD', u'tomatoMeter', u'Writer', u'tomatoUserRating', u'Production', u'Actors', u'tomatoFresh', u'Type', u'imdbVotes', u'Website', u'tomatoConsensus', u'Poster', u'tomatoRotten', u'Director', u'Released', u'tomatoUserReviews', u'Awards', u'Genre', u'tomatoUserMeter', u'imdbRating', u'Language', u'Country', u'BoxOffice', u'Runtime', u'tomatoReviews', u'imdbID', u'Metascore', u'Response', u'tomatoRating', u'Year']


In [78]:
movieHeaders[0]

u'Plot'

In [79]:
for i in range(0, len(r.json().keys())):
  type(r.json().values()[i])

In [80]:
len(r.json().keys())

34

In [81]:
int(r.json().values()[5])

92

In [82]:
print r.json()['Genre']

Action, Adventure, Sci-Fi


In [99]:
MovieID,Title,Genre=[str(r.json()['imdbID']),str(r.json()['Title']),str(r.json()['Genre'])]